In [4]:
import yfinance as yf
import pandas as pd
def fetch_yfinance_data(tickers: list[str], period="1mo", interval="1d") -> pd.DataFrame:
        df = yf.download(tickers, period=period, interval=interval, group_by='ticker', auto_adjust=False)
        flat_df = (
            df.stack(level=0, future_stack=True)
            .reset_index()
            .rename(columns={
                "Date": "date",
                "Open": "open",
                "High": "high",
                "Low": "low",
                "Close": "close",
                "Volume": "volume",
                "Ticker": "ticker"
            })[["date", "ticker", "open", "high", "low", "close", "volume"]]
        )
        return flat_df

In [48]:
TICKERS = [
    "AAPL", "TSLA", "AMZN", "MSFT", "NVDA",
    "GME", "AMC", "META","JPM", "SPY",
    "UNH", "C"
]
df = fetch_yfinance_data(TICKERS, period="1mo", interval="1d")

[*********************100%***********************]  12 of 12 completed


In [49]:
def interpolate_group(x):
    x = x.infer_objects()
    # Convert 'close' to numeric (float), in case it's still object dtype
    x['close'] = pd.to_numeric(x['close'], errors='coerce')
    return x.interpolate()

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['ticker', 'date'])
print(df.columns)
df1 = df.groupby('ticker').apply(interpolate_group).reset_index(drop=True)
print(df1.columns)

df1['returns'] = df1.groupby('ticker')['close'].pct_change()
q_low = df1['returns'].quantile(0.01)
q_high = df1['returns'].quantile(0.99)
df2 = df1[(df1['returns'] > q_low) & (df1['returns'] < q_high)]

Index(['date', 'ticker', 'open', 'high', 'low', 'close', 'volume'], dtype='object', name='Price')
Index(['date', 'ticker', 'open', 'high', 'low', 'close', 'volume'], dtype='object', name='Price')


/var/folders/y9/m6kzn12n3yn2h7b76mwm3zv40000gn/T/ipykernel_64233/1046565517.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return x.interpolate()
/var/folders/y9/m6kzn12n3yn2h7b76mwm3zv40000gn/T/ipykernel_64233/1046565517.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return x.interpolate()
/var/folders/y9/m6kzn12n3yn2h7b76mwm3zv40000gn/T/ipykernel_64233/1046565517.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return x.interpolate()
/var/folders/y9/m6kzn12n3yn2h7b76mwm3zv40000gn/T/ipykernel_64233/1046565517.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in 

In [50]:
df2

Price,date,ticker,open,high,low,close,volume,returns
1,2025-05-01,AAPL,209.080002,214.559998,208.899994,213.320007,57365700,0.003859
2,2025-05-02,AAPL,206.089996,206.990005,202.160004,205.350006,101010600,-0.037362
3,2025-05-05,AAPL,203.100006,204.100006,198.210007,198.889999,69018500,-0.031459
4,2025-05-06,AAPL,198.210007,200.649994,197.020004,198.509995,51216500,-0.001911
5,2025-05-07,AAPL,199.169998,199.440002,193.250000,196.250000,68536700,-0.011385
...,...,...,...,...,...,...,...,...
259,2025-05-23,UNH,296.100006,299.700012,291.640015,295.570007,16873600,-0.003708
260,2025-05-27,UNH,299.500000,299.950012,289.130005,295.000000,20144600,-0.001929
261,2025-05-28,UNH,294.000000,303.779999,293.279999,298.109985,16114200,0.010542
262,2025-05-29,UNH,296.799988,303.380005,295.000000,298.170013,16221100,0.000201


In [51]:
df2['ticker'].value_counts()

ticker
AAPL    21
AMZN    21
C       21
JPM     21
META    21
MSFT    21
NVDA    21
SPY     21
TSLA    21
GME     20
UNH     19
AMC     18
Name: count, dtype: int64

In [53]:
import requests
def fetch_financelayer_news(keywords, limit=100):
    url = "https://api.apilayer.com/financelayer/news"
    headers = {"apikey": "test"}
    params = {
        "keywords": ",".join(keywords),
        "dateFrom": "2025-05-01",
        "dateTo": "2025-05-02",
        "limit": limit
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    articles = data.get("data", [])
    df = pd.DataFrame(articles)
    return df

In [54]:
df3 = fetch_financelayer_news(TICKERS, limit=100)

In [55]:
import re
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = text.lower()
    tokens = text.split()
    tokens = [w for w in tokens if w not in stop_words]
    return " ".join(tokens)

def preprocess_reddit(df: pd.DataFrame) -> pd.DataFrame:
    df['cleaned_text'] = df['title'].fillna('') + ' ' + df['selftext'].fillna('')
    df['cleaned_text'] = df['cleaned_text'].apply(clean_text)
    return df

def preprocess_financelayer(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop_duplicates(subset=['title', 'published_at'])
    df = df.dropna(subset=['title', 'description'])
    df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')
    df['cleaned_text'] = df['title'].fillna('') + ' ' + df['description'].fillna('')
    df['cleaned_text'] = df['cleaned_text'].apply(clean_text)
    return df

In [56]:
df4 = preprocess_financelayer(df3)

In [57]:
df4

,title,url,description,source,tickers,tags,published_at,cleaned_text
0,"Magnificent 7 Earnings: What to Expect (AMZN, ...",https://www.zacks.com/commentary/2459147/magni...,Pardon Our Interruption\nAs you were browsing ...,zacks.com,"[aapl, amzn, meta, meta, metv, msft]","[Consumer Cyclical, ETF, Investment Ideas, Sto...",2025-04-30 14:35:00+00:00,magnificent 7 earnings expect amzn aapl meta m...
1,"SA analyst upgrades: TSLA, AAPL, NVDA, META, P...",https://seekingalpha.com/news/4430194-sa-analy...,Seeking Alpha analysts issue upgrades for TSLA...,seekingalpha.com,[],[],2025-04-10 16:09:50+00:00,sa analyst upgrades tsla aapl nvda meta pltr a...
2,"Earnings week ahead: MSFT, AAPL, META, AMZN, Q...",https://seekingalpha.com/news/4435596-earnings...,Get ready for a pivotal week in the market as ...,seekingalpha.com,[],[],2025-04-27 12:03:33+00:00,earnings week ahead msft aapl meta amzn qcom p...
3,"SA Analyst Rating Downgrades: MSFT, TSLA, NVDA...",https://seekingalpha.com/news/4429498-sa-analy...,Seeking Alpha analysts have issued downgrades ...,seekingalpha.com,"[msft, nvda]","[Stock, Technology]",2025-04-08 16:38:42+00:00,sa analyst rating downgrades msft tsla nvda ii...
4,"Appaloosa trims AMZN, MSFT, QCOM, and adds to ...",https://seekingalpha.com/news/4449323-appaloos...,Discover Appaloosa Management's position moves...,seekingalpha.com,[],[],2025-05-15 23:50:02+00:00,appaloosa trims amzn msft qcom adds nrg glw un...
...,...,...,...,...,...,...,...,...
95,Nvidia (NVDA) Moves 18.7% Higher: Will This St...,https://www.zacks.com/stock/news/2443106/nvidi...,Pardon Our Interruption\nAs you were browsing ...,zacks.com,"[intc, nvda]","[Stock, Tale Of The Tape, Technology]",2025-04-10 12:30:00+00:00,nvidia nvda moves 187 higher strength last par...
96,UnitedHealth unit to cut up to 25% of reauthor...,https://seekingalpha.com/news/4422199-unitedhe...,UnitedHealth's (UNH) Optum Rx simplifies acces...,seekingalpha.com,[unh],"[Healthcare, Stock]",2025-03-19 14:14:59+00:00,unitedhealth unit cut 25 reauthorizations unh ...
97,"Compared to Estimates, AMC Entertainment (AMC)...",https://www.zacks.com/stock/news/2464755/compa...,Pardon Our Interruption\nAs you were browsing ...,zacks.com,[amc],"[Consumer Cyclical, Fundamental Analysis, Stock]",2025-05-08 00:00:06+00:00,compared estimates amc entertainment amc q1 ea...
98,GameStop (GME) Stock Moves -0.04%: What You Sh...,https://www.zacks.com/stock/news/2452169/games...,Pardon Our Interruption\nAs you were browsing ...,zacks.com,[gme],"[Consumer Cyclical, Stock, Tale Of The Tape]",2025-04-21 21:45:32+00:00,gamestop gme stock moves 004 know pardon inter...


In [58]:
df4.tickers.value_counts()

tickers
[gme]                                                  17
[]                                                     16
[msft]                                                  5
[aaic, ai, msft]                                        4
[nvda]                                                  4
[unh]                                                   4
[aapl, msft]                                            3
[tsla]                                                  2
[amc]                                                   2
[aapl, meta, msft]                                      2
[aapl, amzn, meta, meta, metv, msft]                    2
[aapl]                                                  2
[msft, nvda]                                            2
[avxl, nvda]                                            1
[msft, unh]                                             1
[meta, meta, metv]                                      1
[nvda, zs]                                              1
[amc, 

In [59]:
df4.tags.value_counts()

tags
[Consumer Cyclical, Stock, Tale Of The Tape]                                                                                                                         14
[]                                                                                                                                                                   11
[Stock, Tale Of The Tape, Technology]                                                                                                                                 6
[Stock, Stockstory, Technology, Tiingo Top, Why-It-Moves-Single]                                                                                                      5
[Consumer Cyclical, Stock]                                                                                                                                            5
[Stock, Technology]                                                                                                                                        